<a href="https://colab.research.google.com/github/louisbrown1/spanishfriend/blob/main/spanishfriend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-dotenv
!pip install boto3
!pip install openai
!pip install tiktoken
!pip install ipywebrtc

In [ ]:
import pandas as pd
import boto3
import openai
import time
import tiktoken
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file


In [ ]:
openai.api_key  = os.getenv('OPENAI_API_KEY')
aws_key = os.getenv('key_id')
region = os.getenv('region_name')
aws_secret_access_key =os.getenv('secret_key')

In [ ]:
transcribe = boto3.client('transcribe',
                          aws_access_key_id = aws_key, #insert your access key ID here
                          aws_secret_access_key = aws_secret_access_key, # insert your secret access key here
                          region_name = region # region: usually, I put "eu-west-2"
)

In [ ]:
def check_job_name(job_name):
  job_verification = True

  # all the transcriptions
  existed_jobs = transcribe.list_transcription_jobs()

  for job in existed_jobs['TranscriptionJobSummaries']:
    if job_name == job['TranscriptionJobName']:
      job_verification = False
      break

  if job_verification == False:
    command = input(job_name + " has existed. \nDo you want to override the existed job (Y/N): ")
    if command.lower() == "y" or command.lower() == "yes":
      transcribe.delete_transcription_job(TranscriptionJobName=job_name)
    elif command.lower() == "n" or command.lower() == "no":
      job_name = input("Insert new job name? ")
      check_job_name(job_name)
    else:
      print("Input can only be (Y/N)")
      command = input(job_name + " has existed. \nDo you want to override the existed job (Y/N): ")
  return job_name

Single Speaker

In [ ]:
def amazon_transcribe(audio_file_name):
  job_uri = 's3://transcibefun/Untitled.mp3' # your S3 access link
  # Usually, I put like this to automate the process with the file name
  # "s3://bucket_name" + audio_file_name

  # Usually, file names have spaces and have the file extension like .mp3
  # we take only a file name and delete all the space to name the job
  job_name = (audio_file_name.split('.')[0]).replace(" ", "")

  # file format
  file_format = audio_file_name.split('.')[1]

  # check if name is taken or not
  job_name = check_job_name(job_name)
  transcribe.start_transcription_job(
      TranscriptionJobName=job_name,
      Media={'MediaFileUri': job_uri},
      MediaFormat = file_format,
      LanguageCode='en-GB')

  while True:
    result = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    if result['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED','FAILED']:
      print("FAILED")
      break
    time.sleep(15)

  if result['TranscriptionJob']['TranscriptionJobStatus'] == "COMPLETED":
    data = pd.read_json(result['TranscriptionJob']['Transcript']['TranscriptFileUri'])

  return data['results'][1][0]['transcript']

In [ ]:
from ipywebrtc import AudioRecorder, CameraStream

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [ ]:
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -ac 1 -f mp3 Untitled.mp3 -y -hide_banner -loglevel panic

In [ ]:
client = boto3.client("s3",
                      aws_access_key_id = aws_key,
                      aws_secret_access_key = aws_secret_access_key,
                      region_name = region)
client.upload_file("Untitled.mp3", "transcibefun", "Untitled.mp3")

In [ ]:
file = 'Untitled.mp3'
db = amazon_transcribe(file)

In [ ]:
print(db)

**Create openai helper function**

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message["content"]

In [ ]:
prompt = f"""
Translate the following  text to Spanish: \
```{db}```
"""
response_gpt = get_completion(prompt)
print(response_gpt)

**Create Polly Client**

In [ ]:
polly_client = boto3.Session(
                aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret_access_key,
    region_name='eu-west-2').client('polly')

In [ ]:
response = polly_client.synthesize_speech(VoiceId='Lucia',
                OutputFormat='mp3',
                Text = response_gpt)

file = open('speech.mp3', 'wb')
file.write(response['AudioStream'].read())
file.close()

In [ ]:
from IPython.display import Audio
from IPython.core.display import display
def beep():
    display(Audio('speech.mp3', autoplay=True))
beep();